
Workflow

1. Initialize
    - Workspace
    - compute Cluster (to run the pipeline)
    - Environment
    - Experiment
2. Get Input Data
3. Create a .py script to Train & Register Model
4. Configure ScriptRun

##### Step 1: Initializing Setup

In [ ]:
#Initialize Workspace
from azureml.core import Workspace, Datastore, Dataset

ws = Workspace.from_config()

In [ ]:
# Initialize Compute Target
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
compute = 'ML-Pipeline-Cluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=compute)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=300)
    cpu_cluster = ComputeTarget.create(ws, compute, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# creating an environment
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

env_name = 'ENV-SDKv1'

# ---- Create environment using defining packages
#custom_env = Environment('aml-scikit-learn')
#conda_dep = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy','pip', 'pyodbc','sqlalchemy'],
#                                     pip_packages=['azureml-defaults'])

# custom_env.python.conda_dependencies = conda_dep
# register the environment
#custom_env.register(workspace=ws)

# ---- create enviroment using .yaml file
# python_packages = Environment.from_conda_specification(env_name, '../dependencies/conda.yaml')
# # register the environment
# python_packages.register(workspace=ws)

# # calling registered environent
reg_env = Environment.get(ws, env_name)

# create a run config object for the pipeline
pipeline_runconfig = RunConfiguration()

# # use the compute target
pipeline_runconfig.target = compute

# # assigning the run configuration to the envrionment
pipeline_runconfig.environment = reg_env
print('RunConfiguration created')